In [26]:
# from pymilvus import connections
# connections.connect(host='localhost', port='19530', secure=False)
# print(connections.list_connections())

In [27]:
import os
import re
import time
import pickle
import requests
import json
import pandas as pd
import bs4
import tiktoken
import getpass
from tqdm import tqdm
from tqdm.notebook import tqdm
from collections import defaultdict
from IPython.display import clear_output

from langchain.docstore.document import Document
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.embeddings import ClovaXEmbeddings
from langchain_community.chat_models import ChatClovaX
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from pymilvus import connections
from pymilvus import utility
from langchain_community.vectorstores.milvus import Milvus
from ragas import evaluate
from ragas.metrics import context_precision, faithfulness

# from langchain_huggingface.embeddings import HuggingFaceEmbeddings  # 원래 사용하던 허깅페이스 (참고용)

In [28]:
# Milvus 서버 연결
connections.connect(
    alias="default",
    host="localhost",
    port="19530"
)

In [29]:
print(utility.get_server_version())  # 버전
print(connections.has_connection(alias="default"))  # MILVUS 연결확인 

v2.4.0
True


### API 환경설정

In [30]:
# NCP API Key - 클로바X & bgm-3 임베딩 모델

# 환경변수 등록
os.environ["NCP_CLOVASTUDIO_API_KEY"] = getpass.getpass("Test Key 입력: ")

# url
os.environ["NCP_CLOVASTUDIO_API_URL"] = "https://clovastudio.stream.ntruss.com/"

# # 실험용용
# embeddings = ClovaXEmbeddings(model="bge-m3")
# chat = ChatClovaX(model="HCX-003")

### Test

In [31]:
# print(os.environ.get("NCP_CLOVASTUDIO_API_KEY"))

In [32]:
# # 임베딩 test
# sample_text = "임베딩을 테스트합니다."
# embedding_vector = embeddings.embed_query(sample_text)
# print("임베딩 결과:", embedding_vector)

In [33]:
# messages = [
#     (
#         "system",
#         "CLOVA Studio는 HyperCLOVA X 언어 모델을 활용하여 AI 서비스를 손쉽게 만들 수 있는 개발 도구입니다.",
#     ),
#     ("human", "CLOVA Studio가 무엇인가요?"),
# ]
# ai_msg = chat.invoke(messages)
# ai_msg

### 임베딩

### 데이터 (sample : 10000개) -> 데이터 2000개로 

In [36]:
df = pd.read_csv("C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\sampled_data.csv")

#########################
# Original Sample
#########################
df = df.drop(columns=['Unnamed: 0'])
df = df.sample(n=2000, random_state=2025)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 6448 to 5006
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2000 non-null   int64 
 1   분류      1967 non-null   object
 2   제목      2000 non-null   object
 3   부제      1332 non-null   object
 4   저자      2000 non-null   object
 5   발행자     2000 non-null   object
 6   발행일     2000 non-null   object
 7   페이지     2000 non-null   int64 
 8   가격      2000 non-null   int64 
 9   표지      2000 non-null   object
 10  책소개     1978 non-null   object
 11  저자소개    1508 non-null   object
 12  목차      1804 non-null   object
 13  출판사리뷰   1156 non-null   object
 14  추천사     26 non-null     object
dtypes: int64(3), object(12)
memory usage: 250.0+ KB


In [ ]:
#########################
# Original Sample
#########################
df = df.drop(columns=['Unnamed: 0'])
df = df.sample(n=2000, random_state=2025)

In [41]:
#########################
# Arthor Sample
#########################

df = pd.read_csv(r"C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\aiffel_book_2000_author.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0.1      2000 non-null   int64 
 1   Unnamed: 0        2000 non-null   int64 
 2   ISBN              2000 non-null   int64 
 3   분류                1967 non-null   object
 4   제목                2000 non-null   object
 5   부제                1332 non-null   object
 6   저자                2000 non-null   object
 7   발행자               2000 non-null   object
 8   발행일               2000 non-null   object
 9   페이지               2000 non-null   int64 
 10  가격                2000 non-null   int64 
 11  표지                2000 non-null   object
 12  책소개               1978 non-null   object
 13  저자소개              1508 non-null   object
 14  목차                1852 non-null   object
 15  출판사리뷰             1156 non-null   object
 16  추천사               26 non-null     object
 17  processed_auth

In [42]:
#########################
# Category Sample
#########################

df = pd.read_csv(r"C:\Kill_the_RAG\Project\Aiffel_final_project\Code\Data\aiffel_book_semifinal_2000_category.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ISBN     2000 non-null   int64 
 1   분류       1967 non-null   object
 2   제목       2000 non-null   object
 3   부제       1332 non-null   object
 4   저자       2000 non-null   object
 5   발행자      2000 non-null   object
 6   발행일      2000 non-null   object
 7   페이지      2000 non-null   int64 
 8   가격       2000 non-null   int64 
 9   표지       2000 non-null   object
 10  책소개      1978 non-null   object
 11  저자소개     1508 non-null   object
 12  목차       1804 non-null   object
 13  출판사리뷰    1156 non-null   object
 14  추천사      26 non-null     object
 15  cat_num  2000 non-null   int64 
dtypes: int64(4), object(12)
memory usage: 250.1+ KB


In [7]:
# bge-m3 임베딩 모델
ncp_embeddings = ClovaXEmbeddings(model="bge-m3")

# HCX-003 모델 - llm
llm_clova = ChatClovaX(model="HCX-003")

In [8]:
def split_text(text, chunk_size=1000, overlap=100):
    if text is None or pd.isnull(text):
        return [""]
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

In [9]:
# 밀버스는 메타 데이터에 한글을 못 받는다고 한다. 영문 매핑핑
metadata_mapping = {
    'ISBN': 'ISBN',
    '페이지': 'page',
    '가격': 'price',
    '제목': 'title',
    '저자': 'author'
}

vector_doc_mapping = {
    '제목': 'title',
    '분류': 'category',
    '저자': 'author',
    '저자소개': 'author_intro',
    '책소개': 'book_intro',
    '목차': 'table_of_contents',
    '출판사리뷰': 'publisher_review',
    '추천사': 'recommendation'
}

In [10]:
metadata_columns = ['ISBN', '페이지', '가격', '제목', '저자']
vector_doc_columns = ['제목', '분류', '저자','저자소개', '책소개', '목차', '출판사리뷰','추천사']

In [11]:
RAG_DB = []
for index, row in df.iterrows():
    # vector_doc_columns 영문으로 
    doc_text = ""
    for col in vector_doc_columns:
        value = row.get(col, "")
        if pd.isnull(value):
            value = ""
        eng_col = vector_doc_mapping.get(col, col)
        doc_text += f"{eng_col} : {value}\n"
    # 청킹 
    chunks = split_text(doc_text)
    
    # metadata 컬럼 영문으로 
    metadata = {}
    for col in metadata_columns:
        value = row.get(col, None)
        eng_col = metadata_mapping.get(col, col)
        metadata[eng_col] = value

    for chunk in chunks:
        RAG_DB.append({
            'text': chunk,
            'metadata': metadata
        })

# Document 
documents = [
    Document(page_content=entry['text'], metadata=entry['metadata'])
    for entry in RAG_DB
]

In [ ]:
# 임베딩 로드 OR 생성

# Original
embedding_file = "C:\\Kill_the_RAG\\Project\\Aiffel_final_project\\Code\\Data\\ncp_bge_m3_embeddings.pkl"

## Category
# embedding_file = "C:\\Kill_the_RAG\\Project\\Aiffel_final_project\\Code\\Data\\example\\embedding_Category.pkl"

## Arthor
# embedding_file = "C:\\Kill_the_RAG\\Project\\Aiffel_final_project\\Code\\Data\\example\\embedding_Arthor.pkl"


if os.path.exists(embedding_file):
    with open(embedding_file, "rb") as f:
        saved_data = pickle.load(f)
    all_text_embedding_pairs = saved_data["embeddings"]  # (text, embedding) 튜플 
    all_metadata_list = saved_data["metadata"]            # metadata
    print("임베딩 데이터 불러오기")
else:
    all_text_embedding_pairs = []
    all_metadata_list = []
    batch_size = 2000
    for i, doc in enumerate(tqdm(documents, desc="NCP 임베딩(bge-m3) 처리 중", unit="document")):
        text_chunk = doc.page_content
        embedding = None
        while embedding is None:
            try:
                embedding = ncp_embeddings.embed_query(text_chunk)
            except Exception as e:
                print(f"[에러] 문서 {i} 임베딩 실패: {e}")
                if "429" in str(e):
                    print("[경고] 요청 제한 초과. 대기 후 재시도") # 5 초 후, 재시도 처리 
                    time.sleep(5)
                    continue
                embedding = None
        all_text_embedding_pairs.append((text_chunk, embedding))
        all_metadata_list.append(doc.metadata)
        time.sleep(0.8) 
        if (i + 1) % batch_size == 0:
            print("배치 처리 2000건 완료, 5초 대기 중...") # 배치당 5초 
            time.sleep(5)
    with open(embedding_file, "wb") as f:
        pickle.dump({
            "embeddings": all_text_embedding_pairs,
            "metadata": all_metadata_list
        }, f)
    print("저장 완료") 

    ## pkl 별 저장후, 합치는 방법으로.

임베딩 데이터 불러오기


In [13]:
# 만약 저장된 metadata가 원래 영어 매핑된 값이 아니라면, 재매핑 (여기서는 이미 문서 생성 시 영어로 변환된 상태태)
all_metadata_list_mapped = []
for meta in all_metadata_list:
    mapped_meta = { metadata_mapping.get(key, key): value for key, value in meta.items() }
    all_metadata_list_mapped.append(mapped_meta)

# Milvus 벡터 DB 구축 
# Milvus에 연결
connections.connect(alias="default", host="localhost", port="19530")

collection_name = "book_rag_db"
vectorstore = Milvus(
    embedding_function=ncp_embeddings,
    collection_name=collection_name,
    connection_args={"host": "localhost", "port": "19530"},
    auto_id=True
)

# 사전 계산된 임베딩 사용 - test 용
texts = [pair[0] for pair in all_text_embedding_pairs]
embeds = [pair[1] for pair in all_text_embedding_pairs]

# 만약에 매핑이 일치하지 않으면???
# 일치함!

def precomputed_embed_documents(cls, input_texts):
    if input_texts != texts:
        raise ValueError("ERROR : 입력 텍스트 순서가 사전 계산된 임베딩과 일치하지 않음음")
    return embeds

ClovaXEmbeddings.embed_documents = classmethod(precomputed_embed_documents)

# Milvus에 데이터 태우기 - 청크 당 고유번호가 존재.
vectorstore.add_texts(
    texts=texts,
    metadatas=all_metadata_list_mapped,
    embeddings=embeds
)

C:\Users\computer\AppData\Local\Temp\ipykernel_30884\3417791851.py:12: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vectorstore = Milvus(


[456745779904381816,
 456745779904381817,
 456745779904381818,
 456745779904381819,
 456745779904381820,
 456745779904381821,
 456745779904381822,
 456745779904381823,
 456745779904381824,
 456745779904381825,
 456745779904381826,
 456745779904381827,
 456745779904381828,
 456745779904381829,
 456745779904381830,
 456745779904381831,
 456745779904381832,
 456745779904381833,
 456745779904381834,
 456745779904381835,
 456745779904381836,
 456745779904381837,
 456745779904381838,
 456745779904381839,
 456745779904381840,
 456745779904381841,
 456745779904381842,
 456745779904381843,
 456745779904381844,
 456745779904381845,
 456745779904381846,
 456745779904381847,
 456745779904381848,
 456745779904381849,
 456745779904381850,
 456745779904381851,
 456745779904381852,
 456745779904381853,
 456745779904381854,
 456745779904381855,
 456745779904381856,
 456745779904381857,
 456745779904381858,
 456745779904381859,
 456745779904381860,
 456745779904381861,
 456745779904381862,
 456745779904

In [ ]:
# RetrievalQA 체인 
dense_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

dpr_qa_chain = RetrievalQA.from_chain_type(
    llm=llm_clova,
    retriever=dense_retriever,
    return_source_documents=True
)

In [ ]:
# # 프롬프트
# multi_turn_prompt = PromptTemplate(
#     input_variables=["history", "query"],
#     template="""
# [대화 맥락]
# 사용자 대화 내역:
# {history}
# 사용자의 마지막 질문: "{query}"

# [역할]
# 너는 도서관 사서 챗봇이다. 대화를 통해 사용자의 선호도의 파악 정도를 0에서 1 사이의 확률로 표현하라. 해당 점수가 0.8점을 넘긴다면, DB 검색을 진행하라.
# 사용자 선호도가 충분하지 않다면, 검색 점수를 올릴 수 있는 방향으로 구체적인 추가 질문을 생성하라.
#     - 예 
#         - 어느 장르를 좋아하시나요?
#         - 재미있게 읽으셨던 책의 이름은 무엇인가요?
#         - 해당 책을 찾으시는 목적은 무엇인가요?
#         - 해당 분야를 잘 아시나요? (초급자 or 숙련자)

# [출력 항목]
# 1. 검색 확률: 0에서 1 사이의 숫자 (예: 0.55) 
# 2. 기본 검색 쿼리: 사용자 선호도에 기반한 구체적인 검색 쿼리 텍스트 (예: "한국 고전 소설")
# 3. 추가 질문: 필요한 경우 추가 정보 요청, 검색이 진행되었다면, 빈 문자열

# 출력 예시:
# 검색 확률: 0.75
# 기본 검색 쿼리: "한국 고전 소설"
# 추가 질문: "풍자 소설을 좋아하시나요? or 역사적인 우화를 좋아하시나요?"

# [지시]
# - 대화 내용을 사용자 선호도에 기반하여 10~20 단어로 요약한 결과를 통해 위 항목들을 반드시 모두 출력하라.
# - "기본 검색 쿼리"가 비어 있거나 "None"이면 안 된다. 대화를 이어나갈수록 사용자 선호도에 알맞게 변화하여야한다.
# """
# )
# search_query_chain = LLMChain(llm=llm_clova, prompt=multi_turn_prompt)

# final_query_prompt = PromptTemplate(
#     input_variables=["history", "fallback", "score"],
#     template="""
# [대화 요약]
# {history}

# [기본 검색 쿼리]
# {fallback}

# [이전 검색 확률]
# {score}

# 위 내용을 바탕으로, 사용자 선호도를 종합하여 DB 검색에 사용할 최종 검색 쿼리를 생성하라.
# 최종 출력은 반드시 아래 형식을 준수하라:
# 1. 검색 확률: {{score}}
# 2. 검색 쿼리: "{{final_search_query}}"
# 3. 추가 질문: "{{follow_up_question}}"
# 4. 추천 이유: "{{reason}}"

# (추천 이유 정보가 부족하면 "{{reason}}"에는 "추천 이유 정보 없음"이라고 출력하라.)
# 최종 검색 쿼리:
# """
# )
# # Retrieval에 들어갈 최종 query 체인 
# final_query_chain = LLMChain(llm=llm_clova, prompt=final_query_prompt)

In [ ]:
# 멀티챗을 위한 프롬프트트
multi_turn_prompt = PromptTemplate(
    input_variables=["history", "query"],
    template="""
[대화 맥락]
사용자 대화 내역:
{history}
사용자의 최신 질문: "{query}"

[역할 및 목표]
너는 도서관 사서 역할의 챗봇이다. 사용자와의 대화를 통해 선호도를 알아내라. 
- 사용자가 책을 원하는 목적(단순 흥미, 정보 취득, 시험 준비 등)과 사용자의 사전 지식 수준(어린아이/어른/전문가/초보자)과 구체적 요구사항(특정 시대, 사건, 인물 등)을 파악하라.
- 대화가 진행될수록 검색 쿼리는 단순한 쿼리에서 사용자 선호도에 기반한한 보다 구체적인 쿼리로 업데이트 되어야한다.
- 매 턴 대화를 통해 사용자의 선호도를 0에서 1 사이의 숫자로 평가하라.
    - 사용자가 찾는 카테고리를 안다면, 0.15점 증가
    - 사용자가 책을 찾는 이유가 단순 독서인지, 지식 습득인지, 시험 때문인지를 안다면, 0.1점 증가
    - 사용자의 선호도(이해하기 쉬운 책, 특정 분야를 자세하게 설명하는 책, 재미있다고 평이 자자한 책 등)를 안다면 0.15점 증가
    - 이전 대화 history에서 도출된 선호도 관련 정보 및 점수가 유지되고 있음을 감안하여, 새로운 정보가 기존 선호도를 강화하는 방향으로 평가해라.
    - 평가값이 0.8 이상이면, 최종 검색 쿼리 생성으로 바로 넘어가라.
        - 정보가 충분하면 추가 질문은 빈 문자열("")로 남겨라.
    - 0.8 미만이면, 부족한 정보를 보완하기 위한 구체적인 추가 질문을 반드시 포함하라. 
    - 사용자 선호도에 기반해 검색 점수를 높일 수 있는 방향으로 질문을 생성하라. 

[MultiChat 출력 형식 - 반드시 아래 내용만 출력]
1. 검색 확률: {{숫자, 0~1 사이 (예: 0.75)}}
2. 기본 검색 쿼리: "{{검색 쿼리, 예: 흥미로운 조선시대 역사적 사건을 다룬 한국사 서적}}"
3. 추가 질문: "{{추가 질문, 정보 보완 필요 시 구체적으로; 충분하면 빈 문자열}}"

[출력 예시]
검색 확률: 0.75
기본 검색 쿼리: "이해하기 쉬운 쉬운 역사책"
추가 질문: "혹시 어느 시대 역사책을 찾고 있나요?"
"""
)
search_query_chain = LLMChain(llm=llm_clova, prompt=multi_turn_prompt)


# 최종 쿼리 생성 - 검색에 들어갈
final_query_prompt = PromptTemplate(
    input_variables=["history", "fallback", "score"],
    template="""
[전체 대화 요약]
{history}

[이전 기본 검색 쿼리]
{fallback}

[이전 검색 확률]
{score}

[역할 및 목표]
너는 도서관 사서 역할의 챗봇이다. 이제 멀티챗에서 모은 사용자 선호도를 바탕으로, 최종적으로 DB 검색에 바로 들어갈 구체적이고 풍부한 검색 쿼리를 생성해라.
- 최종 검색 쿼리는 대화 맥락을 기반으로 사용자가 원하는 흥미 요소, 그리고 구체적 요구사항(시대, 인물, 사건 등)을 반드시 반영하여 문장 형태로 작성하라.
- 최종 출력은 **반드시 책 제목, 저자, 추천 이유**를 포함하라.
- 정보가 충분하면 추가 질문은 빈 문자열("")로 표기하고, 검색 이후, 제목/저자는 반드시 존재해야한다. 추천 이유가 부족하다면, 부족한 이유를 설명해라. (추천 이유 정보 없음 등)

[최종 출력 형식 - 반드시 아래 내용만 출력]
검색 확률: {score}
검색 쿼리: "{{최종 검색 쿼리}}"
책 제목: "{{책 제목}}"
저자: "{{저자}}"
추천 이유: "{{추천 이유, 정보 부족 시 '추천 이유 정보 없음'}}"

[출력 예시]
검색 확률: "0.75"
검색 쿼리: "흥미로운 조선시대 역사적 사건과 인물을 다룬 한국사 서적"
책 제목: "조선 왕조 실록"
저자: "홍길동"
추천 이유: "조선시대의 정치, 문화, 경제적 요소를 균형 있고 쉽게 반영하여 심층적 역사 이해에 도움을 줄 수 있음"
    
[지시 사항]
- 반드시 위 출력 형식에 맞춰 출력해라.
- 멀티챗을 통해 수집된 사용자 선호도를 최대한 반영하여, 최종 검색 쿼리와 함께 책 제목, 저자, 추천 이유를 구체적으로 작성하라.
"""
)
final_query_chain = LLMChain(llm=llm_clova, prompt=final_query_prompt)

In [146]:
def extract_field(text, field_name):
    pattern = rf"{re.escape(field_name)}\s*:\s*(.*)"
    match = re.search(pattern, text)
    return match.group(1).strip() if match else ""

In [ ]:
MIN_INFO_LENGTH = 10 # 최소 정보 제한 10자 이내면 생성 X

def generate_answer(query):
    result = dpr_qa_chain.invoke(query)
    source_docs = result['source_documents']

    retrieved_isbns = set()
    for doc in source_docs:
        isbn = doc.metadata.get("ISBN")
        if isbn:
            retrieved_isbns.add(isbn)

    aggregated_docs = []
    for isbn in retrieved_isbns:
        book_docs = [doc for doc in documents if doc.metadata.get("ISBN") == isbn]
        if not book_docs:
            continue
        aggregated_text = "\n".join([doc.page_content for doc in book_docs])
        aggregated_docs.append(Document(page_content=aggregated_text, metadata=book_docs[0].metadata))

    formatted_answers = []
    for doc in aggregated_docs:
        metadata = doc.metadata
        # MILVUS : 영어로 바꿔줘야함 
        title = metadata.get("title") or extract_field(doc.page_content, "title")
        author = metadata.get("author") or extract_field(doc.page_content, "author")
        publisher_review = extract_field(doc.page_content, "publisher_review")
        book_intro = extract_field(doc.page_content, "book_intro")

        if publisher_review and book_intro:
            combined_info = publisher_review + "\n" + book_intro
        elif publisher_review:
            combined_info = publisher_review
        elif book_intro:
            combined_info = book_intro
        else:
            combined_info = ""

        if not combined_info or len(combined_info.strip()) < MIN_INFO_LENGTH:
            reason = "추천 이유 정보 없음"
        else:
            reason_prompt = (
                f"다음 정보를 참고하여, 이 책이 추천되는 이유를 간결하고 명확하게 요약해라. "
                f"사용자 선호도에 기반해서 책의 특징이나 강점을 중심으로 설명해라. 만약 선호도를 모르겠다면, '추천 이유 정보 없음'을 출력해라.\n\n정보:\n{combined_info}"
            )
            reason_response = llm_clova.invoke(reason_prompt)
            generated_reason = reason_response.text().strip()
            if not generated_reason or len(generated_reason) < 10 or "추천 이유 정보 없음" in generated_reason:
                reason = "추천 이유 정보 없음"
            else:
                reason = generated_reason

        formatted = f"{title}\n{author}\n추천 이유: {reason}"
        formatted_answers.append(formatted)

    answer = "\n\n".join(formatted_answers)
    return answer, None

user_preferences = defaultdict(list)
log_history = []

In [148]:
def categorize_preference(question, response):
    if "장르" in question or "어떤 책" in question:
        user_preferences["genre"].append(response)
    elif "작가" in question or "좋아하는 작가" in question:
        user_preferences["author"].append(response)
    elif "목적" in question or "이유" in question:
        user_preferences["purpose"].append(response)
    else:
        user_preferences["misc"].append(response)

def robust_parse_llm_response(response_text):
    # 콜론(:)이랑 공백 파싱 방법 : for NCP에서 활용하는 llm model 파싱
    score_match = re.search(r"검색\s*확률\s*[:：]\s*([\d\.]+)", response_text)
    search_score = float(score_match.group(1)) if score_match else None

    query_match = re.search(r"기본\s*검색\s*쿼리\s*[:：]\s*\"([^\"]+)\"", response_text)
    search_query = query_match.group(1).strip() if query_match else None

    follow_match = re.search(r"추가\s*질문\s*[:：]\s*\"([^\"]*)\"", response_text)
    follow_up_question = follow_match.group(1).strip() if follow_match else ""

    return search_score, search_query, follow_up_question

In [ ]:
def search_and_generate_answer(query, query_history):
    while True:
        query_summary = "\n".join(query_history[-5:])
        prompt_vars = {
            "history": query_summary,
            "query": query
        }
        # 초기 검색 결정
        search_decision_dict = search_query_chain.invoke(prompt_vars)
        response_text = search_decision_dict["text"].strip()
        print("\n[🔍 LLM 응답 확인]\n", response_text)

        search_score, base_search_query, follow_up_question = robust_parse_llm_response(response_text)
        print(f"\n[디버그] 파싱 결과: 검색 확률={search_score}, 기본 검색 쿼리='{base_search_query}', 추가 질문='{follow_up_question}'")

        if search_score is None:
            print("\n[LLM 응답 파싱 실패: 추가 정보 필요]")
            extra_info = input("추가 정보를 입력해주세요: ")
            query_history.append(f"사용자(추가): {extra_info}")
            query = f"{query} {extra_info}"
            continue

        if search_score >= 0.8 and base_search_query:
            # 최종 쿼리 생성에서의 검색 확률은 멀티턴에서 그대로 인자 전달하도록 
            final_search_query = final_query_chain.invoke({
                "history": "\n".join(query_history),
                "fallback": base_search_query,
                "score": search_score
            })["text"].strip()
            print(f"\n[최종 검색 쿼리 생성]: {final_search_query}")

            answer, sources = generate_answer(final_search_query)
            if sources:
                book_info = "\n".join([f"- {title}" for title in sources])
                answer_with_info = f"{answer}\n\n[책 정보]\n{book_info}"
                print("\n[책 정보]\n", book_info)
            else:
                answer_with_info = answer
            return answer_with_info

        if follow_up_question:
            print(f"\n[보충 질문: {follow_up_question}]")
            query_history.append(f"AI: {follow_up_question}")
            user_response = input("\n사용자 응답: ")
            query_history.append(f"사용자: {user_response}")
            categorize_preference(follow_up_question, user_response)
            print("\n[사용자 선호도 업데이트 완료!]")
            query = f"{query} {follow_up_question} {user_response}"
            continue

        if search_score < 0.8 or not base_search_query:
            print("\n[검색 확률 낮거나 검색 쿼리 없음: 추가 정보 필요]")
            extra_info = input("추가 정보를 입력해주세요: ")
            query_history.append(f"사용자(추가): {extra_info}")
            query = f"{query} {extra_info}"
            continue

In [150]:
def interactive_multi_turn_qa():
    query_history = []

    while True:
        clear_output(wait=True)
        print("멀티턴 AI 기반 책 추천 시스템 (종료하려면 'quit' 입력)")
        print("-" * 50)

        query = input("질문을 입력하세요: ")

        if query.lower() == 'quit':
            print("\n[대화 저장 중...]")
            log_history.append(query_history)
            print("대화 저장 완료")
            break

        query_history.append(f"사용자: {query}")
        answer = search_and_generate_answer(query, query_history)

        print("\n[AI의 답변]")
        print(answer)

        query_history.append(f"AI: {answer}")

        input("\n-> 계속하려면 Enter를 누르세요...")

def show_log_history():
    print("\n[ 전체 대화 로그]")
    for i, session in enumerate(log_history, 1):
        print(f"\n 대화 세션 {i}:\n")
        print("\n".join(session))
        print("-" * 50)

In [15]:
# 실행
interactive_multi_turn_qa()

NameError: name 'interactive_multi_turn_qa' is not defined

In [158]:
df[df['제목'].str.contains("조선 왕실 이야기", na=False)]

,ISBN,분류,제목,부제,저자,발행자,발행일,페이지,가격,표지,책소개,저자소개,목차,출판사리뷰,추천사
4933,9791170480440,['국내도서 > 역사 > 조선사 > 조선시대 일반'],조선 왕실 이야기,바로 알고 거울삼아야 할,공준원 지음,생각나눔,2020-06-04T15:00:00.000Z,416,15000,https://image.aladin.co.kr/product/24222/70/co...,정사와 비사를 오가는 흥미진진한 역사 이야기! 태조 이성계는 정말 차사를 죽였을까?...,공준원 1941년 김제시에서 태어난 그는 김제 중고등학교를 거쳐 고려대학교 법대를 ...,머리말\r\n \n이성계와 함흥차사 \n-이성계는 차사를 한 사람도 죽이지 않았다\...,제대로 알아두면 쓸모 있는 역사 역사는 무엇을 말하는가? 현대를 살아가는 우리에게 ...,NaN


In [ ]:
# # 대화 히스토리 & 사용자 선호도
# user_preferences = defaultdict(list)
# log_history = []  # 전체 대화 로그 - 옵션 사항 (만약에 동일 사용자가 계속 책을 추천받으려는 경우)

# """
# [대화 맥락]
# 사용자 대화 내역:
# {history}
# 사용자의 최신 질문: "{query}"

# [역할 및 목표]
# 너는 도서관 사서 역할의 챗봇이다. 사용자와의 대화를 통해 선호도를 알아내라. 
# - 사용자가 책을 원하는 목적(단순 흥미, 정보 취득, 시험 준비 등)과 사용자의 사전 지식 수준(어린아이/어른/전문가/초보자)과 구체적 요구사항(특정 시대, 사건, 인물 등)을 파악하라.
# - 대화가 진행될수록 검색 쿼리는 단순한 쿼리에서 사용자 선호도에 기반한한 보다 구체적인 쿼리로 업데이트 되어야한다.
# - 매 턴 대화를 통해 사용자의 선호도를 0에서 1 사이의 숫자로 평가하라.
#     - 사용자가 찾는 카테고리를 안다면, 0.15점 증가
#     - 사용자가 책을 찾는 이유가 단순 독서인지, 지식 습득인지, 시험 때문인지를 안다면, 0.1점 증가
#     - 사용자의 선호도(이해하기 쉬운 책, 특정 분야를 자세하게 설명하는 책, 재미있다고 평이 자자한 책 등)를 안다면 0.15점 증가가
#     - 평가값이 0.8 이상이면, 최종 검색 쿼리 생성으로 바로 넘어가라.
#         - 정보가 충분하면 추가 질문은 빈 문자열("")로 남겨라.
#     - 0.8 미만이면, 부족한 정보를 보완하기 위한 구체적인 추가 질문을 반드시 포함하라. 
#     - 사용자 선호도에 기반해 검색 점수를 높일 수 있는 방향으로 질문을 생성하라. 

# [MultiChat 출력 형식 - 반드시 아래 내용만 출력]
# 1. 검색 확률: {{숫자, 0~1 사이 (예: 0.75)}}
# 2. 기본 검색 쿼리: "{{검색 쿼리, 예: 흥미로운 조선시대 역사적 사건을 다룬 한국사 서적}}"
# 3. 추가 질문: "{{추가 질문, 정보 보완 필요 시 구체적으로; 충분하면 빈 문자열}}"

# [출력 예시]
# 검색 확률: 0.75
# 기본 검색 쿼리: "이해하기 쉬운 쉬운 역사책"
# 추가 질문: "혹시 어느 시대 역사책을 찾고 있나요?"
# """


# """
# [전체 대화 요약]
# {history}

# [이전 기본 검색 쿼리]
# {fallback}

# [이전 검색 확률]
# {score}

# [역할 및 목표]
# 너는 도서관 사서 역할의 챗봇이다. 이제 멀티챗에서 모은 사용자 선호도를 바탕으로, 최종적으로 DB 검색에 바로 들어갈 구체적이고 풍부한 검색 쿼리를 생성해라.
# - 최종 검색 쿼리는 대화 맥락을 기반으로 사용자가 원하는 흥미 요소, 그리고 구체적 요구사항(시대, 인물, 사건 등)을 반드시 반영하여 문장 형태로 작성하라.
# - 최종 출력은 **반드시 책 제목, 저자, 추천 이유**를 포함하라.
# - 정보가 충분하면 추가 질문은 빈 문자열("")로 표기하고, 검색 이후, 제목/저자는 반드시 존재해야한다. 추천 이유가 부족하다면, 부족한 이유를 설명해라. (추천 이유 정보 없음 등)

# [최종 출력 형식 - 반드시 아래 내용만 출력]
# 검색 확률: {score}
# 검색 쿼리: "{{최종 검색 쿼리}}"
# 책 제목: "{{책 제목}}"
# 저자: "{{저자}}"
# 추천 이유: "{{추천 이유, 정보 부족 시 '추천 이유 정보 없음'}}"

# [출력 예시]
# 검색 확률: "0.75"
# 검색 쿼리: "흥미로운 조선시대 역사적 사건과 인물을 다룬 한국사 서적"
# 책 제목: "조선 왕조 실록"
# 저자: "홍길동"
# 추천 이유: "조선시대의 정치, 문화, 경제적 요소를 균형 있고 쉽게 반영하여 심층적 역사 이해에 도움을 줄 수 있음"
    
# [지시 사항]
# - 반드시 위 출력 형식에 맞춰 출력해라.
# - 멀티챗을 통해 수집된 사용자 선호도를 최대한 반영하여, 최종 검색 쿼리와 함께 책 제목, 저자, 추천 이유를 구체적으로 작성하라.
# """
# # 멀티턴 프롬프트 템플릿 llm이 분기별 작동하게 하는 변수를 지정
# multi_turn_prompt = PromptTemplate(
#     input_variables=["history", "query", "impormation", "strategy", "if", "else"],
#     template="""
# 사용자와의 대화 히스토리 :
# {history}

# 사용자의 마지막 질문 :
# "{query}"

# role :
# {impormation}

# 아래 정보를 바탕으로, 너는 사용자 선호도를 명확히 파악하고, 단호하게 검색 조건을 평가하라.
# 반드시 아래 기준에 따라 행동해야 한다.

# 1. 사용자가 책을 찾는 이유를 파악하라.
#    - (1) 단순 독서를 원함.
#    - (2) 정보나 기술 습득을 위해 찾음.
#    - (3) 흥미 또는 취미 생활의 일환으로 찾음.

# 2. 사용자가 찾고자 하는 책에 대한 사전 지식 수준을 평가하라.
#    - (1) 전혀 모름.
#    - (2) 카테고리, 작가, 관련 책 이름 등 일부 정보를 보유.
#    - (3) 구체적으로 찾고 싶은 책이 있음.
#    해당 정보를 간결하게 요약하여 검색 쿼리에 활용해라.

# 3. 사용자의 취향에 적합한 카테고리를 구체적으로 파악하라.
#    아래 항목 중 해당하는 것을 반드시 언급하라.
#    - 소설: 현대, 고전, 판타지, SF, 미스터리, 로맨스, 사회 비판, 심리/철학, 역사, 전쟁 소설 등
#    - 경제/경영: 기업 경영, 주식, 마케팅, 경제학 입문, 창업, 노동 철학 등
#    - 자기계발: 시간관리, 멘탈 관리, 습관 형성, 리더십, 자기 탐색 등
#    - 시/에세이: 철학적, 문학적, 감성, 자전적, 여행 시집 등
#    - 인문/교양: 철학, 역사, 사회, 심리, 종교/명상 등
#    - 취미/실용: 요리, 운동, DIY, 사진, 글쓰기, 음악/예술 등
#    - 어린이/청소년: 그림책, 초등 필독서, 청소년 소설, 과학/탐구, 경제/교양 등
#    반드시 구체적인 키워드를 포함하라.

# {if}
# 사용자 쿼리를 받았을 때 검색을 할 수 있는 기준

# {strategy}
# 아래 기준에 따라 최종 검색 쿼리를 생성하라:
# 1. 대화 내용을 종합하여 검색 수행 확률을 0에서 1 사이의 값으로 평가하라. 점수의 증가는 세밀하게 진행해라. (사용자가 원하는 카테고리를 안다면, 0.15점 증가!! 사용자가 책을 원하는 이유를 안다면, 0.2점 증가!! 등)확률이 0.8 이상이면 반드시 검색을 진행하라.
# 2. 사용자와 멀티챗(사용자 query & LLM 답변)을 10~40 단어로 요약해서 최종 쿼리(query)를 생성해라. 2~5개의 핵심 키워드는 쿼리에 반드시 포함시켜라.
# 3. 요약문에는 반드시 "책 추천 기준"을 반영하라. 예를 들어, "SF 장르 중 AI 관련 최신 베스트셀러" 또는 "역사 사실에 충실한 근현대사 도서"와 같이 구체적으로 기술하라.
# 4. 최종 검색 쿼리를 생성할 때, 반드시 추출된 핵심 키워드를 포함하고, 사용자 목적에 맞게 최종 쿼리를 확정하라.
#     - 기술 서적을 원한다면, 키워드를 적절하게 나열해라.
#     - 소설/자기개발서 등을 원한다면 적절하게 문장 형태로 쿼리를 조합해라.
# {else}
# 현재까지 멀티챗을 종합했을 때, 사용자 선호도를 알 수 없다면, 5번 문항을 명심해라.

# 5. 만약 사용자 정보가 충분하지 않다면, {impormation}과 {strategy}에 기반하여, 검색 점수를 높일 수 있는 방향으로 적절한 보충 질문을 하나 생성하라.
#     - 예시: "관심 있는 도서 카테고리를 명확히 알려줘." 또는 "찾고 있는 책의 목적을 구체적으로 말해줘." 또는 "재미있게 읽었던 책은 뭐야?"
#     - 예시: "(카테고리를 모른다.) : 네가 관심있어하는 도서 카테고리가 뭐야?"
#     - 예시: "(사용자의 정확한 목적을 모른다.) : 네가 찾고 있는 지식 도서는 초급자 용이야? 상급자 용이야?"

# 6. 추천 이유 {{reason}}을 작성할때는 책소개,저자,저자소개,출판사리뷰,추천사 등을 적절하게 활용하도록해.
#     - 추천 이유에 적합한 내용을 보고 generation 해라.

# [출력 형식]
# 최종 출력은 반드시 아래 형식을 준수하라.
# 1. 검색 확률: {{score}}
# 2. 검색 쿼리: "{{final_search_query}}"
# 3. 추가 질문: "{{follow_up_question}}"
# 4. 추천 이유: "{{reason}}"
# """
# )
# search_query_chain = LLMChain(llm=llm_gemini, prompt=multi_turn_prompt)


# final_query_prompt = PromptTemplate(
#     input_variables=["history", "fallback"],
#     template="""
# 지금까지의 대화 내용을 바탕으로, 사용자의 선호도와 요청을 반영하여 검색에 적절한 최종 쿼리를 생성해.
# 1. 대화 내용을 요약하고 핵심 키워드를 추출해.
# 2. 추출된 핵심 키워드를 포함하여, 사용자 목적에 맞는 구체적인 검색 쿼리를 생성해.
# 대화 내용:
# {history}
# 기본 검색 쿼리: {fallback}
# 최종 검색 쿼리:
# """
# )
# final_query_chain = LLMChain(llm=llm_gemini, prompt=final_query_prompt)